In [1]:
import numpy as np
from fxpmath import Fxp
import matplotlib.pyplot as plt
from FPGAJacobi.Cordic import Cordic

In [2]:
class Jacobi:
    def __init__(self, 
                 n_cordic_iterations : int = 12, 
                 n_word : int = 17, 
                 n_frac : int = 15, 
                 is_signed : bool = True,
                 matrix_dim : int = 12):
                 
    
        self._n_word = n_word
        self._n_frac = n_frac
        self._is_signed = is_signed
        self._rotation_cordic = Cordic(mode = "rotation", 
                              n_iterations = n_cordic_iterations, 
                              n_word = self._n_word, 
                              n_frac = self._n_frac, 
                              is_signed = True)
        self._vectoring_cordic = Cordic(mode = "vectoring", 
                              n_iterations = n_cordic_iterations, 
                              n_word = self._n_word, 
                              n_frac = self._n_frac, 
                              is_signed = True)
        self._N = matrix_dim
        self._A = Fxp(np.zeros((_N, _N)), n_word = self._n_word, n_frac = self._n_frac, signed = self._is_signed)
        self._W = Fxp(np.zeros((_N, _N)), n_word = self._n_word, n_frac = self._n_frac, signed = self._is_signed)
    
    def get_A(A : Fxp):
        self._A.equal(A)
    
    def init_W():
        self._W.set_val(np.eye(self._N))
        
    def update_A(i,j):
        _, _, theta = self._vectoring_cordic(A[i,j] << 1, A[i,i] - A[i,j], 0)
        
        #diagonal
        temp_ii, temp_jj, _ = self._rotation_cordic(A[i,i], A[j,j], theta)
        A[i,i], A[j,j], _ = self._rotation_cordic(temp_ii, temp_jj, theta)
        
        #not diagonal i,j
        A[i,j] = A[j,i] = 0
        
        #not diagonal != i,j
        for k in range(self._N):
            if (k != i and k != j):
                A[k,i], A[k,j], _ = self.rotation_cordic(A[k,i], A[k,j], theta)
                A[i,k] = A[k,i]
                A[j,k] = A[k,j]
            
        
        
                 
        
        

SyntaxError: invalid syntax (2266836641.py, line 28)